In [1]:
#The follwoing notebook contains code that takes in FlightRows10000 and prepares for early data anyslsis
#Work done includes removing mostly empty columns, dummies, and fit/transforming.

In [2]:
import pandas as pd

In [3]:
#Short Feature Engineering, remove columns that would not be avaiable until after flight is over. Shortcut by peaking at columns
#avaiable in flights_test
heading=pd.read_csv('lighthouse-labs-midterm/TestRows10forHeading.txt', delimiter='\t')
colList=heading.columns

In [4]:
dfRead = pd.read_csv('lighthouse-labs-midterm/FlightRows10000.txt', delimiter='\t')
dfRead.head(5)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-10-08,WN,WN,WN,2014,WN,N7858A,2014,12892,LAX,...,373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-07-23,DL,DL_CODESHARE,DL,5388,9E,N8969A,5388,10821,BWI,...,255,0.0,7.0,3.0,0.0,46.0,NaN,NaN,NaN,NaN
2,2019-02-28,AS,AS_CODESHARE,AS,3354,OO,N193SY,3354,14831,SJC,...,1450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-06-25,WN,WN,WN,464,WN,N7702A,464,11292,DEN,...,472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-05-01,AS,AS_CODESHARE,AS,2677,QX,N637QX,2677,14679,SAN,...,1080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Get target variable
y=dfRead['arr_delay']

In [6]:
#get predictor col, X
df=dfRead[colList]

In [7]:
#df.select_dtypes(exclude='object')

In [8]:
#check column data type, observer int64, float64, and object
df.dtypes

fl_date               object
mkt_unique_carrier    object
branded_code_share    object
mkt_carrier           object
mkt_carrier_fl_num     int64
op_unique_carrier     object
tail_num              object
op_carrier_fl_num      int64
origin_airport_id      int64
origin                object
origin_city_name      object
dest_airport_id        int64
dest                  object
dest_city_name        object
crs_dep_time           int64
crs_arr_time           int64
dup                   object
crs_elapsed_time       int64
flights                int64
distance               int64
dtype: object

In [9]:
#Check Columns for any empty
df.isnull().sum(axis=0)

fl_date               0
mkt_unique_carrier    0
branded_code_share    0
mkt_carrier           0
mkt_carrier_fl_num    0
op_unique_carrier     0
tail_num              0
op_carrier_fl_num     0
origin_airport_id     0
origin                0
origin_city_name      0
dest_airport_id       0
dest                  0
dest_city_name        0
crs_dep_time          0
crs_arr_time          0
dup                   0
crs_elapsed_time      0
flights               0
distance              0
dtype: int64

In [10]:
#Grab all Numeric columns in preperation for fit transform.
dfNumericValues=df.select_dtypes(exclude='object')
dfNumericValues.head()

,mkt_carrier_fl_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,flights,distance
0,2014,2014,12892,14893,1840,2010,90,1,373
1,5388,5388,10821,14492,1526,1636,70,1,255
2,3354,3354,14831,11259,945,1520,215,1,1450
3,464,464,11292,13871,1350,1615,85,1,472
4,2677,2677,14679,14004,1420,1720,180,1,1080


In [11]:
dfNumericValues.nunique(axis=0, dropna=True)
#Flight useless, should remove
#mkt_carrier_fl_num 7226 in full database
#op_carrier_fl_num 7252
#origin_airport_id 376
#dest_airport_id 376

mkt_carrier_fl_num    4840
op_carrier_fl_num     4841
origin_airport_id      305
dest_airport_id        319
crs_dep_time          1068
crs_arr_time          1191
crs_elapsed_time       386
flights                  1
distance              1286
dtype: int64

In [12]:
#convert crs times, written in 2400 format, into hours by simply moving decimal place over 2 times, ignore errors

dfNumericValues.loc[:,'crs_dep_time']=dfRead.loc[:,'crs_dep_time']//100
dfNumericValues.loc[:,'crs_arr_time']=dfRead.loc[:,'crs_arr_time']//100
dfNumericValues

C:\Users\FBI\anaconda3\lib\site-packages\pandas\core\indexing.py:1047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


,mkt_carrier_fl_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,flights,distance
0,2014,2014,12892,14893,18,20,90,1,373
1,5388,5388,10821,14492,15,16,70,1,255
2,3354,3354,14831,11259,9,15,215,1,1450
3,464,464,11292,13871,13,16,85,1,472
4,2677,2677,14679,14004,14,17,180,1,1080
...,...,...,...,...,...,...,...,...,...
9995,1697,1697,13796,12892,10,12,85,1,337
9996,4005,4005,10397,14574,12,13,86,1,357
9997,4016,4016,12266,13244,12,14,98,1,468
9998,2996,2996,12892,11603,20,22,133,1,748


In [13]:
#first 4 columns are actually categorical columns masked as numeric
dfNumericValues.iloc[:,4:]

,crs_dep_time,crs_arr_time,crs_elapsed_time,flights,distance
0,18,20,90,1,373
1,15,16,70,1,255
2,9,15,215,1,1450
3,13,16,85,1,472
4,14,17,180,1,1080
...,...,...,...,...,...
9995,10,12,85,1,337
9996,12,13,86,1,357
9997,12,14,98,1,468
9998,20,22,133,1,748


In [14]:
#TO do,

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#X=scaler.fit_transform(dfNumericValues.iloc[:,4:]) #TO BE COMPLETED WHEN first 4 columns removed to make categorical
X=scaler.fit_transform(dfNumericValues)
X.shape
#Don't scale y, negative signs important and fit transform will not preserve that property

(10000, 9)

In [16]:
#Number of Unique count in each objects, deciding which are worth grabbing for dummies
(
    df
    .select_dtypes(include='object')
    .nunique(axis=0, dropna=True)
)

fl_date                730
mkt_unique_carrier      11
branded_code_share      16
mkt_carrier             11
op_unique_carrier       28
tail_num              4664
origin                 305
origin_city_name       300
dest                   319
dest_city_name         312
dup                      1
dtype: int64

In [17]:
#Tail num seems useless, but actually in all 16mil, there are 6487 unique values. Consider keeping? But our model will need more data
#Dup has a single value and has no predictive properties. All others seem good
df['tail_num'], df['dup']

(0       N7858A
 1       N8969A
 2       N193SY
 3       N7702A
 4       N637QX
          ...  
 9995    N754SW
 9996    N868CA
 9997    N14993
 9998    N708SK
 9999    N448SW
 Name: tail_num, Length: 10000, dtype: object,
 0       N
 1       N
 2       N
 3       N
 4       N
        ..
 9995    N
 9996    N
 9997    N
 9998    N
 9999    N
 Name: dup, Length: 10000, dtype: object)

In [18]:
dfObjects=(df
    .select_dtypes(include='object')
    .drop(['dup'],1)
)
dfObjects

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,op_unique_carrier,tail_num,origin,origin_city_name,dest,dest_city_name
0,2019-10-08,WN,WN,WN,WN,N7858A,LAX,"Los Angeles, CA",SMF,"Sacramento, CA"
1,2018-07-23,DL,DL_CODESHARE,DL,9E,N8969A,BWI,"Baltimore, MD",RDU,"Raleigh/Durham, NC"
2,2019-02-28,AS,AS_CODESHARE,AS,OO,N193SY,SJC,"San Jose, CA",DAL,"Dallas, TX"
3,2018-06-25,WN,WN,WN,WN,N7702A,DEN,"Denver, CO",OMA,"Omaha, NE"
4,2019-05-01,AS,AS_CODESHARE,AS,QX,N637QX,SAN,"San Diego, CA",PAE,"Everett, WA"
...,...,...,...,...,...,...,...,...,...,...
9995,2018-05-21,WN,WN,WN,WN,N754SW,OAK,"Oakland, CA",LAX,"Los Angeles, CA"
9996,2019-04-07,DL,DL_CODESHARE,DL,OO,N868CA,ATL,"Atlanta, GA",ROA,"Roanoke, VA"
9997,2019-01-28,UA,UA_CODESHARE,UA,EV,N14993,IAH,"Houston, TX",MEM,"Memphis, TN"
9998,2018-11-14,AA,AA_CODESHARE,AA,OO,N708SK,LAX,"Los Angeles, CA",EUG,"Eugene, OR"


In [19]:
# dateTimeConvert=pd.to_datetime(dfObjects['fl_date'], infer_datetime_format=True)
# dateTimeConvert=pd.DatetimeIndex(dateTimeConvert).month
# dateTimeConvert

#one line the above
dfObjects['month']=pd.DatetimeIndex(pd.to_datetime(dfObjects['fl_date'], infer_datetime_format=True)).month
dfObjects

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,op_unique_carrier,tail_num,origin,origin_city_name,dest,dest_city_name,month
0,2019-10-08,WN,WN,WN,WN,N7858A,LAX,"Los Angeles, CA",SMF,"Sacramento, CA",10
1,2018-07-23,DL,DL_CODESHARE,DL,9E,N8969A,BWI,"Baltimore, MD",RDU,"Raleigh/Durham, NC",7
2,2019-02-28,AS,AS_CODESHARE,AS,OO,N193SY,SJC,"San Jose, CA",DAL,"Dallas, TX",2
3,2018-06-25,WN,WN,WN,WN,N7702A,DEN,"Denver, CO",OMA,"Omaha, NE",6
4,2019-05-01,AS,AS_CODESHARE,AS,QX,N637QX,SAN,"San Diego, CA",PAE,"Everett, WA",5
...,...,...,...,...,...,...,...,...,...,...,...
9995,2018-05-21,WN,WN,WN,WN,N754SW,OAK,"Oakland, CA",LAX,"Los Angeles, CA",5
9996,2019-04-07,DL,DL_CODESHARE,DL,OO,N868CA,ATL,"Atlanta, GA",ROA,"Roanoke, VA",4
9997,2019-01-28,UA,UA_CODESHARE,UA,EV,N14993,IAH,"Houston, TX",MEM,"Memphis, TN",1
9998,2018-11-14,AA,AA_CODESHARE,AA,OO,N708SK,LAX,"Los Angeles, CA",EUG,"Eugene, OR",11


In [32]:
dfConcat= pd.concat([dfObjects,dfNumericValues], axis=1)
dfConcat

,fl_date,branded_code_share,mkt_carrier,op_unique_carrier,tail_num,origin,dest,month,mkt_carrier_fl_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,flights,distance
0,2019-10-08,WN,WN,WN,N7858A,LAX,SMF,10,2014,2014,12892,14893,18,20,90,1,373
1,2018-07-23,DL_CODESHARE,DL,9E,N8969A,BWI,RDU,7,5388,5388,10821,14492,15,16,70,1,255
2,2019-02-28,AS_CODESHARE,AS,OO,N193SY,SJC,DAL,2,3354,3354,14831,11259,9,15,215,1,1450
3,2018-06-25,WN,WN,WN,N7702A,DEN,OMA,6,464,464,11292,13871,13,16,85,1,472
4,2019-05-01,AS_CODESHARE,AS,QX,N637QX,SAN,PAE,5,2677,2677,14679,14004,14,17,180,1,1080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2018-05-21,WN,WN,WN,N754SW,OAK,LAX,5,1697,1697,13796,12892,10,12,85,1,337
9996,2019-04-07,DL_CODESHARE,DL,OO,N868CA,ATL,ROA,4,4005,4005,10397,14574,12,13,86,1,357
9997,2019-01-28,UA_CODESHARE,UA,EV,N14993,IAH,MEM,1,4016,4016,12266,13244,12,14,98,1,468
9998,2018-11-14,AA_CODESHARE,AA,OO,N708SK,LAX,EUG,11,2996,2996,12892,11603,20,22,133,1,748


In [33]:
dfConcat.to_csv('lighthouse-labs-midterm/X.csv')
y.to_csv('lighthouse-labs-midterm/y.csv')

In [21]:
#The below are highly highly correlated columns. And some initial exploration
#But they can be dropped at next step. Left behind as examples

In [22]:
dfObjects.groupby(['origin_city_name','origin']).count()

,,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,op_unique_carrier,tail_num,dest,dest_city_name,month
origin_city_name,origin,,,,,,,,,
"Abilene, TX",ABI,2,2,2,2,2,2,2,2,2
"Aguadilla, PR",BQN,3,3,3,3,3,3,3,3,3
"Akron, OH",CAK,9,9,9,9,9,9,9,9,9
"Albany, NY",ALB,18,18,18,18,18,18,18,18,18
"Albuquerque, NM",ABQ,40,40,40,40,40,40,40,40,40
...,...,...,...,...,...,...,...,...,...,...
"Wichita, KS",ICT,12,12,12,12,12,12,12,12,12
"Williston, ND",ISN,1,1,1,1,1,1,1,1,1
"Wilmington, NC",ILM,9,9,9,9,9,9,9,9,9


In [23]:
# Terrible code to find why origin city has 300 vs origin has 305, for curiosity
dfObjects.groupby(['origin_city_name','origin']).count().reset_index().groupby('origin_city_name').count().sort_values('origin')


,origin,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,op_unique_carrier,tail_num,dest,dest_city_name,month
origin_city_name,,,,,,,,,,
"Abilene, TX",1,1,1,1,1,1,1,1,1,1
"Norfolk, VA",1,1,1,1,1,1,1,1,1,1
"Niagara Falls, NY",1,1,1,1,1,1,1,1,1,1
"Newport News/Williamsburg, VA",1,1,1,1,1,1,1,1,1,1
"Newburgh/Poughkeepsie, NY",1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
"Chicago, IL",2,2,2,2,2,2,2,2,2,2
"New York, NY",2,2,2,2,2,2,2,2,2,2
"Houston, TX",2,2,2,2,2,2,2,2,2,2


In [24]:
dfObjects.loc[dfObjects['origin_city_name']=='Chicago, IL']['origin'].unique()

array(['ORD', 'MDW'], dtype=object)

In [25]:
dfObjects.loc[dfObjects['origin_city_name']=='New York, NY']['origin'].unique()

array(['LGA', 'JFK'], dtype=object)

In [26]:
dfObjects.loc[dfObjects['origin_city_name']=='Houston, TX']['origin'].unique()

array(['HOU', 'IAH'], dtype=object)

In [27]:
dfObjects.loc[dfObjects['origin_city_name']=='Phoenix, AZ']['origin'].unique()

array(['PHX', 'AZA'], dtype=object)

In [28]:
dfObjects.loc[dfObjects['origin_city_name']=='Washington, DC']['origin'].unique()

array(['DCA', 'IAD'], dtype=object)

In [29]:
#Keep Code Share, maybe it matters
dfObjects.groupby(['mkt_unique_carrier','branded_code_share']).count()

fl_date  mkt_carrier  \
mkt_unique_carrier branded_code_share                         
AA                 AA                     1158         1158   
                   AA_CODESHARE           1439         1439   
AS                 AS                      333          333   
                   AS_CODESHARE            194          194   
B6                 B6                      370          370   
DL                 DL                     1217         1217   
                   DL_CODESHARE            986          986   
F9                 F9                      158          158   
G4                 G4                      124          124   
HA                 HA                       98           98   
                   HA_CODESHARE             16           16   
NK                 NK                      236          236   
UA                 UA                      789          789   
                   UA_CODESHARE           1184         1184   
VX                 VX                        9            9   
WN                 WN                     1689         1689   

                                       op_unique_carrier  tail_num  origin  \
mkt_unique_carrier branded_code_share                                        
AA                 AA                               1158      1158    1158   
                   AA_CODESHARE                     1439      1439    1439   
AS                 AS                                333       333     333   
                   AS_CODESHARE                      194       194     194   
B6                 B6                                370       370     370   
DL                 DL                               1217      1217    1217   
                   DL_CODESHARE                      986       986     986   
F9                 F9                                158       158     158   
G4                 G4                                124       124     124   
HA                 HA                                 98        98      98   
                   HA_CODESHARE                       16        16      16   
NK                 NK                                236       236     236   
UA                 UA                                789       789     789   
                   UA_CODESHARE                     1184      1184    1184   
VX                 VX                                  9         9       9   
WN                 WN                               1689      1689    1689   

                                       origin_city_name  dest  dest_city_name  \
mkt_unique_carrier branded_code_share                                           
AA                 AA                              1158  1158            1158   
                   AA_CODESHARE                    1439  1439            1439   
AS                 AS                               333   333             333   
                   AS_CODESHARE                     194   194             194   
B6                 B6                               370   370             370   
DL                 DL                              1217  1217            1217   
                   DL_CODESHARE                     986   986             986   
F9                 F9                               158   158             158   
G4                 G4                               124   124             124   
HA                 HA                                98    98              98   
                   HA_CODESHARE                      16    16              16   
NK                 NK                               236   236             236   
UA                 UA                               789   789             789   
                   UA_CODESHARE                    1184  1184            1184   
VX                 VX                                 9     9               9   
WN                 WN                              1689  1689            1689   

                                       m

In [30]:
dfObjects.drop(['origin_city_name','dest_city_name','mkt_unique_carrier'], inplace=True,errors='ignore', axis=1)
dfObjects

,fl_date,branded_code_share,mkt_carrier,op_unique_carrier,tail_num,origin,dest,month
0,2019-10-08,WN,WN,WN,N7858A,LAX,SMF,10
1,2018-07-23,DL_CODESHARE,DL,9E,N8969A,BWI,RDU,7
2,2019-02-28,AS_CODESHARE,AS,OO,N193SY,SJC,DAL,2
3,2018-06-25,WN,WN,WN,N7702A,DEN,OMA,6
4,2019-05-01,AS_CODESHARE,AS,QX,N637QX,SAN,PAE,5
...,...,...,...,...,...,...,...,...
9995,2018-05-21,WN,WN,WN,N754SW,OAK,LAX,5
9996,2019-04-07,DL_CODESHARE,DL,OO,N868CA,ATL,ROA,4
9997,2019-01-28,UA_CODESHARE,UA,EV,N14993,IAH,MEM,1
9998,2018-11-14,AA_CODESHARE,AA,OO,N708SK,LAX,EUG,11
